# Minimal indexing Pipeline

In this example, we are creating a very minimal indexing pipeline which uses `notion-haystack` to get Notion pages and write them to a document store.

_We are using `nest_asyncio` below because notion exporter makes use of asyncio, which causes issues with async jobs running in colab._

In [ ]:
!pip install nest-asyncio

import nest_asyncio

nest_asyncio.apply()

In [2]:
import getpass

notion_api_key = getpass.getpass("Enter Notion API key:")

In [ ]:
# This is a minimal example that exports Notion pages to Haystack Documents.

from notion_haystack import NotionExporter

exporter = NotionExporter(api_token=notion_api_key)
exported_pages = exporter.run(page_ids=["your_page_id"])

In [4]:
# This is a minimal example that exports Notion pages to Haystack Documents and writes them to a document store.

from haystack import Pipeline

from notion_haystack import NotionExporter
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()
exporter = NotionExporter(api_token=notion_api_key)
splitter = DocumentSplitter()
writer = DocumentWriter(document_store=document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=exporter, name="exporter")
indexing_pipeline.add_component(instance=splitter, name="splitter")
indexing_pipeline.add_component(instance=writer, name="writer")

indexing_pipeline.connect("exporter.documents", "splitter.documents")
indexing_pipeline.connect("splitter", "writer")

In [ ]:
indexing_pipeline.run(data={"exporter": {"page_ids": ["your_page_id"] }})